In [851]:
import random
import pandas as pd
import numpy as np

In [852]:
# inputs
num_tipos_pacientes = 10
num_recursos = 12
num_areas_pop = 30
num_hospitais = 10
size_square = 100

horizonte_planejado = 13

# valores maximos
estadia_max = 6 #round(horizonte_planejado/2)
qtd_pacientes_por_tipo_area_max = 400
qtd_recursos_por_hospital_max = 250
qtd_recursos_por_hospital_min = 240
distancia_max = 50
data_saida_pac_init_max = estadia_max - 1
qtd_pacientes_liberados_init_max = qtd_recursos_por_hospital_max/2


In [853]:
# tipo de paciente vs recursos usados

tipo_paciente = ["TP" + str(i) for i in range(1, num_tipos_pacientes + 1)]
recursos =  ["R" + str(i) for i in range(1, num_recursos + 1)]
usa_recurso = np.random.choice(['0', '1'], size=(num_tipos_pacientes*num_recursos))

tipo_paciente_df_in = np.tile(tipo_paciente, num_recursos)
tipo_paciente_df = sorted(tipo_paciente_df_in)
recursos_df = np.tile(recursos, num_tipos_pacientes)

#print(tipo_paciente)
#print(recursos)
#print(usa_recurso)

paciente_vs_recurso = pd.DataFrame({'Tipo_de_paciente': tipo_paciente_df, 'Recurso': recursos_df, 'Usa_recurso': usa_recurso})
paciente_vs_recurso['Usa_recurso'] = pd.to_numeric(paciente_vs_recurso['Usa_recurso'])
  
#display(paciente_vs_recurso.sort_values(by=['Tipo_de_paciente', 'Recurso']))

#fazer check se para cada tipo de paciente tem pelo menos 1 recurso sendo alocado
for tipo_paciente_in in paciente_vs_recurso['Tipo_de_paciente'].unique():
  filtro_tipo = paciente_vs_recurso['Tipo_de_paciente'] == tipo_paciente_in
 
  soma_recursos = paciente_vs_recurso.loc[filtro_tipo, 'Usa_recurso'].sum()
  
  if soma_recursos == 0:

      filtro_recurso_1 = paciente_vs_recurso['Recurso'] == 'R1'
      
      paciente_vs_recurso.loc[filtro_tipo & filtro_recurso_1, 'Usa_recurso'] = 1

paciente_vs_recurso.sort_values(by=['Tipo_de_paciente', 'Recurso'])

,Tipo_de_paciente,Recurso,Usa_recurso
0,TP1,R1,1
9,TP1,R10,0
10,TP1,R11,1
11,TP1,R12,0
1,TP1,R2,0
...,...,...,...
112,TP9,R5,1
113,TP9,R6,0
114,TP9,R7,1
115,TP9,R8,0


In [854]:
# tipo de paciente vs tempo de estadia
amostras_normais = np.random.normal(loc=estadia_max / 2, scale=estadia_max / 5, size=num_tipos_pacientes)
amostras_limite = np.clip(np.round(amostras_normais), 1, estadia_max)
amostras_limite = [max(elem, 0)  for elem in amostras_limite]
estadia = amostras_limite #np.random.randint(1, estadia_max, size=num_tipos_pacientes)

#print(tipo_paciente)
#print(estadia)

tipo_paciente_estadia_df = pd.DataFrame({'Tipo_de_paciente': tipo_paciente, 'Estadia': estadia})
display(tipo_paciente_estadia_df)

,Tipo_de_paciente,Estadia
0,TP1,2.0
1,TP2,3.0
2,TP3,4.0
3,TP4,3.0
4,TP5,3.0
5,TP6,2.0
6,TP7,3.0
7,TP8,3.0
8,TP9,4.0
9,TP10,3.0


In [855]:
# tipo de paciente vs área populacional vs dia de entrada vs qtd pacientes
area_pop = ["AP" + str(i) for i in range(1, num_areas_pop + 1)]
dias_planejados = [int(i) for i in range(1, horizonte_planejado + 1)] 
tipo_paciente = ["TP" + str(i) for i in range(1, num_tipos_pacientes + 1)]


qtd_pacientes_normais = np.random.normal(
    loc=qtd_pacientes_por_tipo_area_max / 2, 
    scale=qtd_pacientes_por_tipo_area_max/5, 
    size=num_tipos_pacientes * num_areas_pop * horizonte_planejado
)
qtd_pacientes_limite = np.clip(np.round(qtd_pacientes_normais), 0, qtd_pacientes_por_tipo_area_max)
qtd_pacientes_limite = [max(elem, 0) for elem in qtd_pacientes_limite]

qtd_pacientes = qtd_pacientes_limite #np.random.randint(0, qtd_pacientes_por_tipo_area_max, size=num_tipos_pacientes*num_areas_pop*horizonte_planejado)

tipo_paciente_df = np.tile(tipo_paciente, num_areas_pop*horizonte_planejado)

area_pop_df_in = np.tile(area_pop, num_tipos_pacientes*horizonte_planejado)
area_pop_df = sorted(area_pop_df_in)

dias_planejados_df_in1 = np.tile(dias_planejados, num_areas_pop)

dias_planejados_df_in2 = sorted(dias_planejados_df_in1)

dias_planejados_df = np.tile(dias_planejados_df_in2, num_tipos_pacientes)

area_pop_qtd_pac_df = pd.DataFrame({'Area_pop': area_pop_df, 'Tipo_de_paciente': tipo_paciente_df, 'Qtd_pacientes': qtd_pacientes, 'Dia': dias_planejados_df})
display(area_pop_qtd_pac_df)

,Area_pop,Tipo_de_paciente,Qtd_pacientes,Dia
0,AP1,TP1,130.0,1
1,AP1,TP2,79.0,1
2,AP1,TP3,192.0,1
3,AP1,TP4,165.0,1
4,AP1,TP5,237.0,1
...,...,...,...,...
3895,AP9,TP6,185.0,13
3896,AP9,TP7,180.0,13
3897,AP9,TP8,239.0,13
3898,AP9,TP9,128.0,13


In [856]:
# hospitais vs recursos vs quantidade de recursos
hospitais = ["H" + str(i) for i in range(1, num_hospitais + 1)]

media = (qtd_recursos_por_hospital_min + qtd_recursos_por_hospital_max) / 2
desvio_padrao = (qtd_recursos_por_hospital_max - qtd_recursos_por_hospital_min) / 6
qtd_recursos_normais = np.random.normal(
    loc=media, 
    scale=desvio_padrao, 
    size=num_recursos * num_hospitais
)
qtd_recursos_limite = np.clip(np.round(qtd_recursos_normais), qtd_recursos_por_hospital_min, qtd_recursos_por_hospital_max)
qtd_recursos_limite = [max(elem,0) for elem in qtd_recursos_limite]

qtd_recursos = qtd_recursos_limite # np.random.randint(qtd_recursos_por_hospital_min, qtd_recursos_por_hospital_max, size=num_recursos*num_hospitais)

hospitais_df = np.tile(hospitais, num_recursos)
recursos_df_in = np.tile(recursos, num_hospitais)
recursos_df = sorted(recursos_df_in)
#print(hospitais_df)
#print(qtd_recursos)

hospitais_vs_rec_qtd_df = pd.DataFrame ({'Hospital': hospitais_df, 'Recurso': recursos_df, 'Qtd_recurso': qtd_recursos})
display(hospitais_vs_rec_qtd_df.sort_values(by=['Hospital','Recurso']))

,Hospital,Recurso,Qtd_recurso
0,H1,R1,244.0
10,H1,R10,247.0
20,H1,R11,244.0
30,H1,R12,248.0
40,H1,R2,244.0
...,...,...,...
78,H9,R5,246.0
88,H9,R6,246.0
98,H9,R7,243.0
108,H9,R8,247.0


In [857]:
# Função para calcular a distância euclidiana
def calcular_distancia(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# Função principal
def calcular_distancias(hospitais, area_pop):
    # Dicionários para armazenar as localizações dos hospitais e áreas populacionais
    localizacao_hospitais = {}
    localizacao_area_pop = {}
    
    # Atribuir localizações aleatórias no plano cartesiano (x, y entre 0 e 100)
    for hospital in hospitais:
        localizacao_hospitais[hospital] = (random.randint(0, size_square), random.randint(0, size_square))
        
    for area in area_pop:
        localizacao_area_pop[area] = (random.randint(0, size_square), random.randint(0, size_square))
    
    # Criar lista para armazenar as distâncias
    distancias = []
    
    # Calcular as distâncias euclidianas entre cada hospital e cada área populacional
    for hospital in hospitais:
        for area in area_pop:
            dist = calcular_distancia(localizacao_hospitais[hospital], localizacao_area_pop[area])
            distancias.append([hospital, area, round(dist, 2)])  # Arredonda a distância para 2 casas decimais
    
    # Criar DataFrame
    df = pd.DataFrame(distancias, columns=["Hospital", "Area_pop", "Distancia"])
    
    return df


In [858]:
# hospitais vs area pop vs distância (gerando aleatoriamente pontos num plano cartesiano)
hospitais_vs_area_pop_dist_df = calcular_distancias(hospitais, area_pop)
hospitais_vs_area_pop_dist_df

,Hospital,Area_pop,Distancia
0,H1,AP1,53.82
1,H1,AP2,61.52
2,H1,AP3,44.01
3,H1,AP4,65.60
4,H1,AP5,49.04
...,...,...,...
295,H10,AP26,65.60
296,H10,AP27,46.57
297,H10,AP28,38.42
298,H10,AP29,71.87


In [859]:
# # hospitais vs area pop vs distância
# distancia = np.random.randint(1, distancia_max, size=num_hospitais*num_areas_pop)

# hospitais_dist_df = np.tile(hospitais, num_areas_pop)
# area_pop_dist_df_in = np.tile(area_pop, num_hospitais)
# area_pop_dist_df = sorted(area_pop_dist_df_in)

# hospitais_vs_area_pop_dist_df = pd.DataFrame ({'Hospital': hospitais_dist_df, 'Area_pop': area_pop_dist_df, 'Distancia': distancia})
# display(hospitais_vs_area_pop_dist_df.sort_values(by=['Hospital','Area_pop']))

In [860]:
area_pop

['AP1',
 'AP2',
 'AP3',
 'AP4',
 'AP5',
 'AP6',
 'AP7',
 'AP8',
 'AP9',
 'AP10',
 'AP11',
 'AP12',
 'AP13',
 'AP14',
 'AP15',
 'AP16',
 'AP17',
 'AP18',
 'AP19',
 'AP20',
 'AP21',
 'AP22',
 'AP23',
 'AP24',
 'AP25',
 'AP26',
 'AP27',
 'AP28',
 'AP29',
 'AP30']

In [861]:
# hospitais vs area pop vs distância
num_areas_pop

30

In [862]:
# condição inicial: hospitais vs tipo paciente vs qtd vs data de saída
data_saida = np.random.randint(1, data_saida_pac_init_max, size=num_hospitais*num_tipos_pacientes)
qtd_pacientes_liberados_init = np.random.randint (0, qtd_pacientes_liberados_init_max, size = num_hospitais*num_tipos_pacientes)

print(hospitais)
print(tipo_paciente)
print(qtd_pacientes_liberados_init)
print(data_saida)

hospitais_lib_df_in = np.tile(hospitais, num_tipos_pacientes)
hospitais_lib_df = sorted(hospitais_lib_df_in)
tipo_paciente_lib_df = np.tile(tipo_paciente, num_hospitais)

cond_init_df = pd.DataFrame({'Hospital': hospitais_lib_df, 'Tipo_de_paciente': tipo_paciente_lib_df, 'Qtd_pacientes_liberados': qtd_pacientes_liberados_init, 'Dia': data_saida})
display(cond_init_df)

['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10']
['TP1', 'TP2', 'TP3', 'TP4', 'TP5', 'TP6', 'TP7', 'TP8', 'TP9', 'TP10']
[ 24  67  43 103  79  88  76 124  13   8  59  71  30 113  16 120  63  59
  98 108  61  29  91   5  13  36  95  74  53  92  30   8  72  59  60 115
 101  55 111 115  71  80  76 109 107  94  64  43 115  39  94  46  89 117
 108  25 106  69  81  19  46  32  50  43  79  65  34  95  42 116   4  75
  47  89  74  63  41  77  15  54  63  67  69  39  47  95  25 117  84  82
  52 117   8  82  61   7 117 124  99  82]
[2 1 2 1 3 3 4 2 4 1 3 1 1 1 2 2 3 2 4 1 3 4 1 1 2 3 1 2 4 2 4 1 1 2 4 1 1
 1 4 4 4 3 4 2 2 3 4 4 4 2 4 3 2 4 4 2 4 4 2 1 3 3 4 4 2 3 1 3 1 2 2 2 4 1
 2 4 3 1 4 3 1 2 1 3 3 3 4 3 2 3 4 2 3 1 2 3 2 1 2 2]


,Hospital,Tipo_de_paciente,Qtd_pacientes_liberados,Dia
0,H1,TP1,24,2
1,H1,TP2,67,1
2,H1,TP3,43,2
3,H1,TP4,103,1
4,H1,TP5,79,3
...,...,...,...,...
95,H9,TP6,7,3
96,H9,TP7,117,2
97,H9,TP8,124,1
98,H9,TP9,99,2


In [863]:
paciente_vs_recurso.to_csv('../data/paciente_vs_recurso.csv', index=False)
tipo_paciente_estadia_df.to_csv('../data/tipo_paciente_estadia.csv', index=False)
area_pop_qtd_pac_df.to_csv('../data/area_pop_qtd_pacientes.csv', index=False)
hospitais_vs_rec_qtd_df.to_csv('../data/hospitais_vs_recursos_qtd.csv', index=False)
hospitais_vs_area_pop_dist_df.to_csv('../data/hospitais_vs_area_pop_dist.csv', index=False)
cond_init_df.to_csv('../data/cond_init.csv', index=False)